In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import itertools
from tqdm import tqdm
import numba
from numba import jit

## Import the datasets

In [43]:
df = pd.read_csv('data/Xtr1.csv')
df2 = pd.read_csv('data/Xte1.csv')
df["Id"] = range(2000)
df = df.set_index("Id")
df2["Id"] = 1000 + df2["Id"]
df2 = df2.set_index("Id")
df = pd.concat((df, df2), axis=0)

NB: We are only interested in $K(Xtr, Xtr)$ and $K(Xtr, Xte)$.

## Substring Kernel

All sequences is this set have length 101.

We have two parameters $k$ and $\lambda$.

Let $u \in A^k$. If $i=(i_1, ..., i_k)$, let $x(i)=x_{i_1}...x_{i_k}$. We define :
$\Phi_u(x) = \sum_{i, x(i) = u} \lambda^{i_k - i_1 + 1}$

In [44]:
k = 4
lambd = 0.7
letters = ["A", "C", "G", "T"]
index = np.arange(85)
for i in range(4):
    index[ord(letters[i])] = i

The dot product is computed recursively using auxiliary quantities defined p. 353 and 356.

In [45]:
def one_hot(s):
    """
    Input : string
    Output : n*4 0/1 matrix
    """
    result = np.zeros((len(s), 4))
    array = np.array(list(s))
    for i in range(4):
        result[:, i] = (array == letters[i])
    return result

Pre-computes the one-hot encodings of the DNA sequences :

In [46]:
one_hots = np.zeros((df.shape[0], len(df["seq"][0]), 4))
for i in range(df.shape[0]):
    one_hots[i] = one_hot(df["seq"][i])

df_int = one_hots.argmax(axis=2) # letters replaced with integers

Auxiliary function $B$ :

In [47]:
@jit(nopython=True)
def compute_B(x, y, k, lambd):
    """
    Input : x, y are strings
    Returns a n*n*k tensor B such that B[l,i,j] = B_l(x[0:i+1], y[0:j+1])
    """
    n = len(x)
    B = np.zeros((k+1, n, n))
    B[0, :, :] = 1
    for l in range(1, k+1):
        if l > n:
            break
        for i in range(l-1, n):
            for j in range(l-1, n):
                a = x[i]
                b = y[j]
                if i > 0:
                    B[l, i, j] += lambd * B[l, i-1, j]
                if j > 0:
                    B[l, i, j] += lambd * B[l, i, j-1]
                if i > 0 and j > 0:
                    B[l, i, j] -= lambd**2 * B[l, i-1, j-1]
                    if a == b:
                        B[l, i, j] += lambd**2 * B[l-1, i-1, j-1]
    return B

In [48]:
@jit(nopython=True)
def compute_B(x, y, k, lambd):
    """
    Input : x, y are strings
    Returns a n*n*k tensor B such that B[l,i,j] = B_l(x[0:i+1], y[0:j+1])
    """
    n = len(x)
    B = np.zeros((k+1, n, n))
    B[0, :, :] = 1

    for l in range(1, k+1):
        if l > n:
            break
        for i in range(l-1, n):
            for j in range(l-1, n):
                a = x[i]
                b = y[j]
                if i > 0:
                    B[l, i, j] += lambd * B[l, i-1, j]
                if j > 0:
                    B[l, i, j] += lambd * B[l, i, j-1]
                if i > 0 and j > 0:
                    B[l, i, j] -= lambd**2 * B[l, i-1, j-1]
                    if a == b:
                        B[l, i, j] += lambd**2 * B[l-1, i-1, j-1]
    return B

In [49]:
x = "ACGT"
y = "TGCG"

In [50]:
compute_B(x, y, 2, lambd)

array([[[1.     , 1.     , 1.     , 1.     ],
        [1.     , 1.     , 1.     , 1.     ],
        [1.     , 1.     , 1.     , 1.     ],
        [1.     , 1.     , 1.     , 1.     ]],

       [[0.     , 0.     , 0.     , 0.     ],
        [0.     , 0.     , 0.49   , 0.343  ],
        [0.     , 0.49   , 0.686  , 0.9702 ],
        [0.     , 0.343  , 0.4802 , 0.67914]],

       [[0.     , 0.     , 0.     , 0.     ],
        [0.     , 0.     , 0.     , 0.     ],
        [0.     , 0.     , 0.     , 0.2401 ],
        [0.     , 0.     , 0.     , 0.16807]]])

In [51]:
%%timeit
_ = compute_B(df["seq"][0], df["seq"][1], k, lambd)

11.1 ms ± 313 µs per loop (mean ± std. dev. of 7 runs, 100 loops each)


In [52]:
def compute_K_words(x, y, xo, yo, k, lambd):
    """
    df_int is the list of integer sequences
    Returns a n*n*k tensor K such that K[l,i,j] = K_l(x[0:i], y[0:j])
    """
    n = len(x)
    B = compute_B(x, y, k, lambd)
    K = np.zeros((k+1, n, n))
    K[0, :, :] = 1
    
    for l in range(1, k+1):
        if l > n:
            break
        for i in range(l-1, n):
            for j in range(l-1, n):
                a = x[i]
                if i > 0:
                    K[l, i, j] = K[l, i-1, j]
                    s = 0
                    mask = yo[1:j+1, index[ord(a)]]
                    K[l, i, j] += lambd**2 * mask.dot(B[l-1, i-1, :j])
    return K

In [53]:
@jit(nopython=True)
def compute_K(df_int, one_hots, i, j, k, lambd):
    """
    df_int is the list of integer sequences
    Returns a n*n*k tensor K such that K[l,i,j] = K_l(x[0:i], y[0:j])
    """
    x = df_int[i]
    y = df_int[j]
    xo = one_hots[i]
    yo = one_hots[j]
    
    n = df_int.shape[1]
    B = compute_B(x, y, k, lambd)
    K = np.zeros((k+1, n, n))
    K[0, :, :] = 1
    
    for l in range(1, k+1):
        if l > n:
            break
        for i in range(max(l-1,1), n): # we skip i = 0, which yields K = 0
            for j in range(l-1, n):
                a = x[i]
                K[l, i, j] = K[l, i-1, j]
                s = 0
                mask = yo[1:j+1, a]
                K[l, i, j] += lambd**2 * mask.dot(B[l-1, i-1, :j])
    return K

In [54]:
x = "GGTT"
y = "TGTA"
xo = one_hot(x)
yo = one_hot(y)
compute_K_words(x, y, xo, yo, k, lambd)

array([[[1.     , 1.     , 1.     , 1.     ],
        [1.     , 1.     , 1.     , 1.     ],
        [1.     , 1.     , 1.     , 1.     ],
        [1.     , 1.     , 1.     , 1.     ]],

       [[0.     , 0.     , 0.     , 0.     ],
        [0.     , 0.49   , 0.49   , 0.49   ],
        [0.     , 0.49   , 0.98   , 0.98   ],
        [0.     , 0.49   , 1.47   , 1.47   ]],

       [[0.     , 0.     , 0.     , 0.     ],
        [0.     , 0.     , 0.     , 0.     ],
        [0.     , 0.     , 0.2401 , 0.2401 ],
        [0.     , 0.     , 0.40817, 0.40817]],

       [[0.     , 0.     , 0.     , 0.     ],
        [0.     , 0.     , 0.     , 0.     ],
        [0.     , 0.     , 0.     , 0.     ],
        [0.     , 0.     , 0.     , 0.     ]],

       [[0.     , 0.     , 0.     , 0.     ],
        [0.     , 0.     , 0.     , 0.     ],
        [0.     , 0.     , 0.     , 0.     ],
        [0.     , 0.     , 0.     , 0.     ]]])

In [55]:
%%timeit
_ = compute_K(df_int, one_hots, 0, 1, k, lambd)

9.96 ms ± 1.06 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


## Application

In [59]:
@jit(nopython=True)
def kernel(df_int, one_hots, i, j, k, lambd):
    return compute_K(df_int, one_hots, i, j, k, lambd)[-1, -1, -1]

In [60]:
K_Xtr_Xtr = np.zeros((2000, 2000))

In [61]:
K_Xtr_Xte = np.zeros((2000, 1000))

In [32]:
for i in tqdm(range(2000)):
    K_Xtr_Xtr[i, 0] = kernel(df_int, one_hots, i, 0, k, lambd)

100%|██████████| 2000/2000 [00:12<00:00, 162.77it/s]


In [62]:
def loop1():
    for i in tqdm(range(2000)):
        for j in range(i+1):
            K_Xtr_Xtr[i, j] = kernel(df_int, one_hots, i, j, k, lambd)
            K_Xtr_Xtr[j, i] = K_Xtr_Xtr[i, j]

In [63]:
def loop2():
    for i in tqdm(range(2000)):
        for j in range(1000):
            K_Xtr_Xte[i, j] = kernel(df_int, one_hots, i, j+2000, k, lambd)

In [64]:
loop1()

100%|██████████| 2000/2000 [5:33:02<00:00, 19.31s/it]


In [34]:
def f(i, j):
    return kernel(df_int, one_hots, i, j, k, lambd)

In [43]:
K = [[f(i, j) for i in range(50)] for j in range(50)]
K = np.array(K)

In [65]:
pd.DataFrame(K_Xtr_Xtr).to_csv("data/Xtr1_substring4_0.7.csv", header=False, index=False, sep=" ")

In [46]:
eg = np.linalg.eig(K_Xtr_Xtr+1e-10*np.eye(2000))

In [53]:
(eg[0] > 0).all()

True

In [35]:
np.linalg.norm(K_Xtr_Xtr-K_Xtr_Xtr.T)

0.0